In [7]:
import argparse
import torch
import torch.nn as nn
import torch.nn.parallel
import glob 
import numpy as np
import sobel

In [8]:
import cv2
from PIL import Image
# from tensorboard_logger import configure, log_value
import pandas as pd
import os
import csv
import re

In [9]:
from models import modules, net, resnet, densenet, senet
import loaddata
import util

In [34]:
def define_model(is_resnet, is_densenet, is_senet):
    if is_resnet:
        original_model = resnet.resnet50(pretrained = True)
        Encoder = modules.E_resnet(original_model) 
        model = net.model(Encoder, num_features=2048, block_channel = [256, 512, 1024, 2048])
    if is_densenet:
        original_model = densenet.densenet161(pretrained=True)
        Encoder = modules.E_densenet(original_model)
        model = net.model(Encoder, num_features=2208, block_channel = [192, 384, 1056, 2208])
    if is_senet:
        original_model = senet.senet154(pretrained=None)
        Encoder = modules.E_senet(original_model)
        model = net.model(Encoder, num_features=2048, block_channel = [256, 512, 1024, 2048])

    return model

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [36]:
x="pretrain/Block0_skip_model_110.pth.tar"
model = define_model(is_resnet=False, is_densenet=False, is_senet=True)
# model = torch.nn.DataParallel(model,device_ids=[0]).cuda()
model.to(device)
state_dict = torch.load(x)['state_dict']

del state_dict["E.Harm.dct"]
del state_dict["E.Harm.weight"]
del state_dict["E.Harm.bias"]

model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
test_loader = loaddata.getTestingData(3, "dataset/test_osi1.csv")

In [ ]:
def testing_loss(depth , output, losses, batchSize):
    
    ones = torch.ones(depth.size(0), 1, depth.size(2),depth.size(3)).float().cuda()
    get_gradient = sobel.Sobel().cuda()
    cos = nn.CosineSimilarity(dim=1, eps=0)
    depth_grad = get_gradient(depth)
    output_grad = get_gradient(output)
    depth_grad_dx = depth_grad[:, 0, :, :].contiguous().view_as(depth)
    depth_grad_dy = depth_grad[:, 1, :, :].contiguous().view_as(depth)
    output_grad_dx = output_grad[:, 0, :, :].contiguous().view_as(depth)
    output_grad_dy = output_grad[:, 1, :, :].contiguous().view_as(depth)
    depth_normal = torch.cat((-depth_grad_dx, -depth_grad_dy, ones), 1)
    output_normal = torch.cat((-output_grad_dx, -output_grad_dy, ones), 1)

    loss_depth = torch.log(torch.abs(output - depth) + 0.5).mean()

    loss_dx = torch.log(torch.abs(output_grad_dx - depth_grad_dx) + 0.5).mean()
    loss_dy = torch.log(torch.abs(output_grad_dy - depth_grad_dy) + 0.5).mean()
    loss_normal = torch.abs(1 - cos(output_normal, depth_normal)).mean()
    loss = loss_depth + loss_normal + (loss_dx + loss_dy)
    losses.update(loss.data, batchSize)

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]  


In [ ]:
def test(test_loader, model):
    losses = AverageMeter()
    model.eval()
    model.cuda()
    totalNumber = 0
    errorSum = {'MSE': 0, 'RMSE': 0, 'MAE': 0,'SSIM':0}

    for i, sample_batched in enumerate(test_loader):
        image, depth = sample_batched['image'], sample_batched['depth']
        depth = depth.cuda()
        image = image.cuda()
        
        with torch.no_grad():
            output = model(image)

        output = torch.nn.functional.interpolate(output,size=(440,440),mode='bilinear')


        batchSize = depth.size(0)
        testing_loss(depth,output,losses,batchSize)


        totalNumber = totalNumber + batchSize

       

        errors = util.evaluateError(output, depth,i,batchSize)

        errorSum = util.addErrors(errorSum, errors, batchSize)
        averageError = util.averageErrors(errorSum, totalNumber)
                     

    averageError['RMSE'] = np.sqrt(averageError['MSE'])
    loss = float((losses.avg).data.cpu().numpy())



    print('Model Loss {loss:.4f}\t'
        'MSE {mse:.4f}\t'
        'RMSE {rmse:.4f}\t'
        'MAE {mae:.4f}\t'
        'SSIM {ssim:.4f}\t'.format(loss=loss,mse=averageError['MSE']\
            ,rmse=averageError['RMSE'],mae=averageError['MAE'],\
            ssim=averageError['SSIM']))



In [ ]:
test(test_loader, model)

In [47]:
imagefile="./data/test_rgbs_base/315000_233500_RGB_0_4.tif"
from PIL import Image
image = Image.open(imagefile)

In [2]:
def centerCrop(image, size):

    w1, h1 = image.size

    tw, th = size

    if w1 == tw and h1 == th:
        return image

    x1 = int(round((w1 - tw) / 2.))
    y1 = int(round((h1 - th) / 2.))

    image = image.crop((x1, y1, tw + x1, th + y1))

    return image

In [48]:
image=centerCrop(image,[440,440])

In [49]:
pic=image
image=torch.ByteTensor(
                torch.ByteStorage.from_buffer(pic.tobytes()))

In [50]:
nchannel = len(pic.mode)
image = image.view(pic.size[1], pic.size[0], nchannel)
image = image.transpose(0, 1).transpose(0, 2).contiguous()
image = image.float()/255

In [51]:
image

tensor([[[0.7059, 0.6941, 0.7137,  ..., 0.4980, 0.4314, 0.5216],
         [0.7529, 0.6667, 0.4314,  ..., 0.5569, 0.5490, 0.4745],
         [0.7294, 0.6706, 0.5608,  ..., 0.3647, 0.4824, 0.5804],
         ...,
         [0.6353, 0.6510, 0.5608,  ..., 0.4000, 0.2275, 0.0980],
         [0.6157, 0.6314, 0.6078,  ..., 0.1804, 0.1412, 0.0588],
         [0.5961, 0.5804, 0.5765,  ..., 0.1294, 0.1059, 0.0941]],

        [[0.6980, 0.6941, 0.7255,  ..., 0.4431, 0.4078, 0.4510],
         [0.7451, 0.6549, 0.4353,  ..., 0.5176, 0.4902, 0.4314],
         [0.7294, 0.6824, 0.5569,  ..., 0.4078, 0.5020, 0.5412],
         ...,
         [0.6000, 0.6314, 0.5451,  ..., 0.3569, 0.2118, 0.1059],
         [0.6039, 0.6235, 0.5843,  ..., 0.1843, 0.1412, 0.0745],
         [0.5412, 0.5255, 0.5373,  ..., 0.1255, 0.1098, 0.1216]],

        [[0.7098, 0.6980, 0.7216,  ..., 0.4392, 0.4157, 0.4431],
         [0.7333, 0.6588, 0.4667,  ..., 0.4941, 0.4706, 0.4431],
         [0.7216, 0.6863, 0.5882,  ..., 0.3804, 0.4667, 0.

In [52]:
__imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                    'std': [0.229, 0.224, 0.225]}

In [53]:
for t, m, s in zip(image, __imagenet_stats['mean'], __imagenet_stats['std']):
    t.sub_(m).div_(s)
image    

tensor([[[ 0.9646,  0.9132,  0.9988,  ...,  0.0569, -0.2342,  0.1597],
         [ 1.1700,  0.7933, -0.2342,  ...,  0.3138,  0.2796, -0.0458],
         [ 1.0673,  0.8104,  0.3309,  ..., -0.5253, -0.0116,  0.4166],
         ...,
         [ 0.6563,  0.7248,  0.3309,  ..., -0.3712, -1.1247, -1.6898],
         [ 0.5707,  0.6392,  0.5364,  ..., -1.3302, -1.5014, -1.8610],
         [ 0.4851,  0.4166,  0.3994,  ..., -1.5528, -1.6555, -1.7069]],

        [[ 1.0805,  1.0630,  1.2031,  ..., -0.0574, -0.2150, -0.0224],
         [ 1.2906,  0.8880, -0.0924,  ...,  0.2752,  0.1527, -0.1099],
         [ 1.2206,  1.0105,  0.4503,  ..., -0.2150,  0.2052,  0.3803],
         ...,
         [ 0.6429,  0.7829,  0.3978,  ..., -0.4426, -1.0903, -1.5630],
         [ 0.6604,  0.7479,  0.5728,  ..., -1.2129, -1.4055, -1.7031],
         [ 0.3803,  0.3102,  0.3627,  ..., -1.4755, -1.5455, -1.4930]],

        [[ 1.3502,  1.2980,  1.4025,  ...,  0.1476,  0.0431,  0.1651],
         [ 1.4548,  1.1237,  0.2696,  ...,  0

In [54]:
image = image[None,:].cuda()

with torch.no_grad():
    output = model(image)
    
output = torch.nn.functional.interpolate(output,size=(440,440),mode='bilinear')    

In [55]:
output

tensor([[[[ 1.9710e-01,  1.9915e-01,  2.0325e-01,  ..., -2.4351e-04,
           -4.5214e-04, -5.5645e-04],
          [ 1.9978e-01,  2.0144e-01,  2.0476e-01,  ..., -1.4563e-04,
           -3.3420e-04, -4.2849e-04],
          [ 2.0513e-01,  2.0601e-01,  2.0777e-01,  ...,  5.0143e-05,
           -9.8321e-05, -1.7255e-04],
          ...,
          [ 1.1790e-01,  1.1857e-01,  1.1989e-01,  ...,  6.4433e-04,
            7.3726e-04,  7.8373e-04],
          [ 1.1517e-01,  1.1611e-01,  1.1799e-01,  ...,  8.5819e-04,
            1.2413e-03,  1.4329e-03],
          [ 1.1380e-01,  1.1488e-01,  1.1703e-01,  ...,  9.6512e-04,
            1.4934e-03,  1.7575e-03]]]], device='cuda:0')

In [56]:
output_0_1 = output.cpu().detach().numpy()
output_0_1= output_0_1*100
output_0_1[np.where(output_0_1>=30)] = 0

In [57]:
output_0_1.shape

(1, 1, 440, 440)

In [58]:
import rasterio
from rasterio.windows import Window

In [61]:
with rasterio.open(imagefile) as src:
    xsize, ysize = 440, 440
    
    xoff, yoff = (src.width - xsize)//2, (src.height - ysize)//2
    
    window=Window(xoff, yoff, xsize, ysize)
    transform = src.window_transform(window)
    
    profile = src.profile
    profile.update({
        'height': ysize,
        'width': xsize,
        'transform': transform
    })
    
    with rasterio.open('./test/image1.tif', 'w', **profile) as dst:
        dst.write(src.read(window=window))

In [88]:
testimage='./test/image2.tif'
with rasterio.open(testimage) as src:
    img=src.read()[0:3]
    img=img.astype(float)/255
    
img=torch.from_numpy(img)

In [89]:
for t, m, s in zip(img, __imagenet_stats['mean'], __imagenet_stats['std']):
    t.sub_(m).div_(s)
img=img.float()   

In [90]:
img = img[None,:].cuda()

with torch.no_grad():
    pred = model(img)
    
pred = torch.nn.functional.interpolate(pred,size=(440,440),mode='bilinear')
pred = pred.cpu().detach().numpy()
pred= pred*100
pred[np.where(pred>=30)] = 0

In [93]:
with rasterio.open(testimage) as src:
    profile = src.profile
    profile.update({
        'count': 1,
        'dtype': rasterio.float32
    })
    
    with rasterio.open('./test/height2.tif','w', **profile) as dst:
        dst.write(pred[0][0], 1)